# Tiny Problem: Ingredients to Named Entities

As a toy problem, we'll see of we can train a model to translate from `ingredients` to `ner`, which should be fairly easy since `ner` is a substring of `ingredients`.

### Some important notes:
- Huggingface `tokenizer`s [produce **only tokens, not embeddings**](https://stats.stackexchange.com/questions/485553/word-embeddings-pre-trained-tokenizers-vs-more-involved-methods). It is on us to re-learn them or find matching embeddings from somewhere (an un-trained tokenizer likely has known, uploaded embeddings).

## Notes on below, this is borked and likely misguided. Read NLP w/ PyTorch book

In [19]:
import sys, random, tqdm, math
# hack to add our own files
if "../" not in sys.path:
    sys.path.insert(0, "../")
import data_processing
from transformers import BertTokenizerFast, AutoModelForSeq2SeqLM, TrainingArguments, Trainer

In [2]:
pretokenized_pairs = data_processing.generated_pretokenized_ingredient_ner_pair_dataset()

Reusing dataset recipe_nlg_lite (/Users/bking/.cache/huggingface/datasets/recipe_nlg_lite/1.0.0/1.0.0/2fd5f76dc1ed88ff2d6485b11497d6ae9516f4ebb2a6cb528dfaf0520bd8e51a)
Loading cached processed dataset at /Users/bking/.cache/huggingface/datasets/recipe_nlg_lite/1.0.0/1.0.0/2fd5f76dc1ed88ff2d6485b11497d6ae9516f4ebb2a6cb528dfaf0520bd8e51a/cache-da15714cbc546e01.arrow


In [3]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')
tokenized_pairs = pretokenized_pairs.map(lambda examples: tokenizer(examples['ingredient'], padding="max_length", truncation=True), batched=True)
tokenized_pairs

  0%|          | 0/63 [00:00<?, ?ba/s]

Dataset({
    features: ['attention_mask', 'ingredient', 'input_ids', 'ner', 'token_type_ids'],
    num_rows: 62731
})

### Fine-tuning the smallest BERT model for this problem

As one baseline for us to compare against (which we will certainly not beat), fine-tune smallest plug-and-play BERT from hugginface `transformers` on this probelem.

In [12]:
model = AutoModel.from_pretrained("bert-base-cased")


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


transformers.models.bert.modeling_bert.BertModel

In [13]:
type(model)

transformers.models.bert.modeling_bert.BertModel

In [17]:
model
# Start with all defaults, only mandatory argument is a directory in which to save checkpoints
training_args = TrainingArguments("test_trainer")

In [20]:
trainer = Trainer(
    model=model, args=training_args, train_dataset=tokenized_pairs, eval_dataset=tokenized_pairs
)

In [21]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertModel.forward` and have been ignored: ner, ingredient.
***** Running training *****
  Num examples = 62731
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 23526


KeyError: 'loss'